In [ ]:
import torch
print(torch.__version__)

# AUTO ENCODER

### 1-1. Simple AUTO ENCODER

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import numpy as np
from tqdm.auto import tqdm

오토인코더는 비지도학습이므로 y의 값을 받지않고 진행합니다.

데이터를 정규화 하여 학습에 알맞은 형태로 바꿉니다.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # 정규화 ?
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

Hidden_input은 아래 그림에서 중간 노란색 노드의 개수를 의미합니다.   
즉, Encoding 단계에서 이미지를 얼마만큼 압축할 것인지를 의미합니다.  

In [ ]:
hidden_dim = 100 # 압축 크기

오토인코더의 구조는 간단합니다.  
입력층 --> 히든층 --> 출력층 으로 이루어져 있습니다.  

선언하는 Model은 원래는 한개지만 압축 후에는 어떤 형태를 이루는지 보기위해서 인코더 모델을 하나 더 만들어보겠습니다.

In [ ]:
# 오토인코더
class SimpleAutoEncoder(nn.Module):
    def __init__(self):
        super(SimpleAutoEncoder, self).__init__()

        self.encoder = nn.Sequential( # 입력, 히든층
            nn.Linear(28*28, hidden_dim) # 인코더에서는 이미지를 압축합니다.
        )

        self.decoder = nn.Sequential( # 출력층
            nn.Linear(hidden_dim, 28*28), # 디코더에서는 원래크기로 복원합니다.
            nn.Sigmoid(),  # 픽셀당 0과 1사이의 값으로 출력하는 sigmoid 활성화 함수를 추가합니다.
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
model = SimpleAutoEncoder()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

criterion = nn.BCELoss().cuda() # sigmoid를 사용했으니 Binary_crossentropy를 사용합니다. 평균제곱오차인 MSELoss도 사용이 가능합니다 !
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)


In [ ]:
# 모델 학습 (tensorflow는 fit 하나로 아래의 과정을 처리한다.)
epochs = 5

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    train_loss = []
    for batch_idx, (data, _) in enumerate(tqdm(iter(train_loader))):
        # 해당 코드가 이미지를 압축했다가 재구성을 하므로 입력과 출력은 data를 넣고 진행합니다.
        # x(입력)와 y(대상 레이블)모두 원본이미지(x)인 것을 주의해야 합니다.
        x = data.view(-1, 28*28).to(device)
        y = data.view(-1, 28*28).to(device)

        optimizer.zero_grad()

        # decoded와 원본이미지(y) 사이의 오차를 구합니다
        encoded, decoded = model(x)
        loss = criterion(decoded, y)

        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    _train_loss = np.mean(train_loss)

    # 테스트
    model.eval()
    val_loss = []
    with torch.no_grad():
        for data, _ in tqdm(iter(test_loader)):
            x = data.view(-1, 28*28).to(device)
            y = data.view(-1, 28*28).to(device)
            encoded, decoded = model(x)
            loss = criterion(decoded, y)
            val_loss.append(loss.item())

    _val_loss = np.mean(val_loss)

    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

    scheduler.step()

이번에는 학습이 잘 되었는지를 확인하기 위해 Predict를 하겠습니다.  
encoder에 test_data를 넣어 압축된 이미지를 생성합니다.  

In [ ]:
model.eval()
with torch.no_grad():
    for data, _ in test_loader:
        data = data.view(-1, 28*28).to(device)
        encoded, decoded = model(data)
        break

원래 이미지를 50개 정도 출력을 해보겠습니다.

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)
        plt.imshow(data[i].reshape(28,28))
        plt.axis('off')
    break

plt.show()

이번에는 압축시킨 이미지를 출력시켜보겠습니다.  

In [ ]:
encoded[0].shape

In [ ]:
plt.figure(figsize=(6,3))
for i in range(50) :
  plt.subplot(5,10,i+1)
  plt.imshow(encoded[i].cpu().reshape(10,10))
  plt.axis('off')
plt.show()

모델에서 압축된 이미지(encoded)와 예측한 이미지(decoded)까지 모두 반환하기 때문에, 예측한 이미지를 이미 가지고 있습니다.

In [ ]:
decoded[3].shape

In [ ]:
plt.figure(figsize=(10,5))
for i in range(50) :
  plt.subplot(5,10,i+1)
  plt.imshow(decoded[i].cpu().reshape(28,28))
  plt.axis('off')
plt.show()

원래 데이터와 압축후에 나온 데이터를 비교했을 때 흐릿한 것을 알 수가 있습니다.  

### 1-2 Deep AUTO ENCODER 실습

이번에는 싱글로 구성된 오토인코더 아닌 여러층으로 깊이 쌓은 오토인코더를 만들어보겠습니다.   
(train_loader, test_loader 부분까지는 위에 코드를 그대로 사용하므로, 생략하겠습니다.)

이번에는 여러층을 쌓기 위해서 hidden_input을 다양하게 설정하겠습니다.

In [ ]:
# 오토인코더 모델 구성
class SimpleAutoEncoder(nn.Module):
    def __init__(self):
        super(SimpleAutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128), # 인코더 부분에서는 차원을 점차 줄여나가겠습니다.
            nn.Linear(128, 64),
            nn.Linear(64, 32),
            nn.Linear(32, 16),
            nn.Linear(16, 4),
        )

        self.decoder = nn.Sequential(
            nn.Linear(4, 16), # 디코더 부분에서는 차원을 점차 28*28로 복원합니다.
            nn.Linear(16, 32),
            nn.Linear(32, 64),
            nn.Linear(64, 128),
            nn.Linear(128, 28*28),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
model = SimpleAutoEncoder()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

criterion = nn.MSELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
# 모델 학습 (tensorflow는 fit 하나로 아래의 과정을 처리한다.)
epochs = 5

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    train_loss = []
    for batch_idx, (data, _) in enumerate(tqdm(iter(train_loader))):
        # 해당 코드가 이미지를 압축했다가 재구성을 하므로 입력과 출력은 data를 넣고 진행합니다.
        # x(입력)와 y(대상 레이블)모두 원본이미지(x)인 것을 주의해야 합니다.
        x = data.view(-1, 28*28).to(device)
        y = data.view(-1, 28*28).to(device)

        optimizer.zero_grad()

        # decoded와 원본이미지(y) 사이의 평균제곱오차를 구합니다
        encoded, decoded = model(x)
        loss = criterion(decoded, y)

        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    _train_loss = np.mean(train_loss)

    # 테스트
    model.eval()
    val_loss = []
    with torch.no_grad():
        for data, _ in tqdm(iter(test_loader)):
            x = data.view(-1, 28*28).to(device)
            y = data.view(-1, 28*28).to(device)
            encoded, decoded = model(x)
            loss = criterion(decoded, y)
            val_loss.append(loss.item())

    _val_loss = np.mean(val_loss)

    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

    scheduler.step()

In [ ]:
model.eval()
with torch.no_grad():
    for data, _ in test_loader:
        data = data.view(-1, 28*28).to(device)
        encoded, decoded = model(data)
        break

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)
        plt.imshow(data[i].reshape(28,28))
        plt.axis('off')
    break

plt.show()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(50) :
  plt.subplot(5,10,i+1)
  plt.imshow(decoded[i].cpu().reshape(28,28))
  plt.axis('off')

plt.show()

### 1-3 Convolutional AUTO ENCODER 실습

활성함수를 relu로 설정하였고, 필터사이즈가 영향을 가지 않게 padding을 설정하였습니다.  

In [ ]:
# 컨볼루션 오토인코더 모델 구성
class CNNAutoEncoder(nn.Module):
    def __init__(self):
        super(CNNAutoEncoder, self).__init__()

        # 활성함수를 relu로 설정하였고, 필터사이즈가 영향을 가지 않게 padding을 설정하였습니다.
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=2, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=2, stride=2, padding=2),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 64, kernel_size=2, stride=2, padding=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2, padding=2),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


모델을 확인 했을 때 이미지가 원래 크기로 돌아온 것을 확인할 수 있습니다.

In [ ]:
%pip install torchsummary

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = CNNAutoEncoder()
model.to(device)

In [ ]:
import torchsummary
torchsummary.summary(model, (1, 28, 28))

In [ ]:
criterion = nn.BCELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [ ]:
# 모델 학습 (tensorflow는 fit 하나로 아래의 과정을 처리한다.)
epochs = 10

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    train_loss = []
    for batch_idx, (data, _) in enumerate(tqdm(iter(train_loader))):
        # 입력이 3차원 벡터이기 때문에, 따로 reshape을 하지 않습니다.
        x = data.to(device)
        y = data.to(device)

        optimizer.zero_grad()

        encoded, decoded = model(x)
        loss = criterion(decoded, y)

        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    _train_loss = np.mean(train_loss)

    # 테스트
    model.eval()
    val_loss = []
    with torch.no_grad():
        for data, _ in tqdm(iter(test_loader)):
            x = data.to(device)
            y = data.to(device)
            encoded, decoded = model(x)
            loss = criterion(decoded, y)
            val_loss.append(loss.item())

    _val_loss = np.mean(val_loss)

    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

    scheduler.step()

예측을 하여 원래 데이터셋과 비교를 해보겠습니다.

In [ ]:
model.eval()
with torch.no_grad():
    for data, _ in test_loader:
        data = data.to(device)
        encoded, decoded = model(data)
        break

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)
        plt.imshow(data[i].permute(1, 2, 0))
        plt.axis('off')
    break

plt.show()

In [ ]:
plt.figure(figsize=(10,5))
for i in range(50) :
  plt.subplot(5,10,i+1)
  plt.imshow(decoded[i].cpu().permute(1, 2, 0))
  plt.axis('off')

plt.show()

CNN 모델을 사용한 AUTO ENCODER를 확인 했을 때 노이즈가 별로 없으며 원래 이미지와 별 차이가 없는 모습을 보입니다.

### 1-4 Denoising AUTO ENCODER

노이즈가 있는 이미지를 AUTO ENCODER를 이용하여 원복 시키겠습니다.  
이번에는 Convolution AUTO ENCODER 코드를 가져와서 사용해보겠습니다.

먼저, 노이즈가 적용되기 전 데이터셋을 살펴 보겠습니다.

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)
        plt.imshow(data[i].permute(1, 2, 0))
        plt.axis('off')
    break

plt.show()

여기에 강제로 노이즈를 추가해보겠습니다.

In [ ]:
np.random.normal(0.0, 1.0, 5)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)

        image_noise = np.random.normal(0.0, 1.0, data[i].shape)
        noise_data = np.clip(data[i]+image_noise*0.4, 0., 1.)

        plt.imshow(noise_data.permute(1, 2, 0))
        plt.axis('off')
    break

plt.show()

노이즈가 적용되서 원래의 이미지보다 알아보기가 힘든 점을 볼수 가 있습니다.  
아까 1-1-3 실습을 할 때 만들었던 코드를 수정없이 진행을 해보겠습니다.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = CNNAutoEncoder()
model.to(device)

criterion = nn.BCELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.005)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

이번에는 결과값에다가는 노이즈가 적용이 안된 데이터셋을 넣어줍니다.  
그래야 노이즈를 제거하고 데이터를 원복 시킬 수 있습니다.  

In [ ]:
# 모델 학습 (tensorflow는 fit 하나로 아래의 과정을 처리한다.)
epochs = 10

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    train_loss = []
    for batch_idx, (data, _) in enumerate(tqdm(iter(train_loader))):
        # 입력 이미지에 임의로 noise를 추가합니다.
        image_noise = torch.rand(data.shape)
        x = torch.clip(data+image_noise*0.4, 0., 1.).to(device)
        y = data.to(device)

        optimizer.zero_grad()

        encoded, decoded = model(x)
        loss = criterion(decoded, y)

        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

    _train_loss = np.mean(train_loss)

    # # 테스트
    # model.eval()
    # val_loss = []
    # with torch.no_grad():
    #     for data, _ in tqdm(iter(test_loader)):
    #         x = data.to(device)
    #         y = data.to(device)
    #         encoded, decoded = model(x)
    #         loss = criterion(decoded, y)
    #         val_loss.append(loss.item())

    # _val_loss = np.mean(val_loss)

    # print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

    scheduler.step()

노이즈 테스트 데이터 셋을 넣어 예측이 잘되는지 살펴보겠습니다.  

In [ ]:
model.eval()
with torch.no_grad():
    for data, _ in test_loader:
        image_noise = torch.rand(data.shape)
        noise_test_data = torch.clip(data+image_noise*0.4, 0., 1.).to(device)
        encoded, decoded = model(noise_test_data)
        break

In [ ]:
# 노이즈 처리된 이미지
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)
        plt.imshow(noise_test_data[i].cpu().permute(1, 2, 0))
        plt.axis('off')
    break

plt.show()

In [ ]:
# decode한 이미지
plt.figure(figsize=(10,5))
for i in range(50) :
  plt.subplot(5,10,i+1)
  plt.imshow(decoded[i].cpu().permute(1, 2, 0))
  plt.axis('off')

plt.show()

확인 결과 노이즈가 제거된 이미지를 확인 할 수 있습니다.

In [ ]:
# 원본
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
for data, _ in test_loader:
    for i, dt in enumerate(data[:50]):
        plt.subplot(5, 10,i+1)
        plt.imshow(data[i].permute(1, 2, 0))
        plt.axis('off')
    break

plt.show()